In [1]:
import torch
from pathlib import Path
import os
from ultralytics import YOLO
from ultralytics.models.yolo.model import DetectionModel # Ensure this import is correct

In [2]:
data_path = os.path.join(os.getcwd(), 'datasets', 'split', '4', 'data.yaml')

In [ ]:
# --- 3. Fine-Tuning ---
print("\n--- Starting Fine-Tuning ---")

# Load the model with transferred weights
model = YOLO('yolo11m.pt') # Load the result of the transfer

# --- Train Model ---
results = model.train(
    # ** Essential Paths & Config **
    data=str(data_path),                # Path to your dataset YAML file
    name='test',                 # Name for the training run directory
    exist_ok=False,                # Error if run name already exists
    save=True,                     # Save checkpoints and final model
    pretrained=True,
    
    # ** Requested Hyperparameters **
    epochs=100,                    # Number of training epochs
    batch=12,                       # Batch size
    imgsz=640,                     # Input image size (height=width=640)

    # ** Data Augmentation Control (ONLY MOSAIC ENABLED) **
    augment=False,                  # MUST be True to enable the augmentation pipeline for mosaic
)


--- Starting Fine-Tuning ---
New https://pypi.org/project/ultralytics/8.3.127 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080, 9877MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=test4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_bu

train: Scanning /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/train/labels... 600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 600/600 [00:00<00:00, 4752.45it/s]

train: New cache created: /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/train/labels.cache



val: Scanning /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/valid/labels... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<00:00, 3125.91it/s]

val: New cache created: /home/ai-laboratory/Documents/TelU-Jurnal-TrendPadaVisiKomputer-FootballPlayerDetection/datasets/split/4/valid/labels.cache


...

      9/100      8.65G      1.296     0.6304     0.9228        370        640:  55%|█████▌    | 21/38 [00:06<00:04,  3.69it/s]

      9/100      8.62G      1.302     0.6307     0.9244        401        640:  61%|██████    | 23/38 [00:06<00:04,  3.53it/s]

      9/100      8.59G      1.301     0.6297     0.9238        392        640:  63%|██████▎   | 24/38 [00:07<00:04,  3.24it/s]

      9/100      8.64G      1.302     0.6289     0.9244        308        640:  68%|██████▊   | 26/38 [00:07<00:03,  3.38it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 9.65 GiB of which 24.75 MiB is free. Including non-PyTorch memory, this process has 8.98 GiB memory in use. Of the allocated memory 8.20 GiB is allocated by PyTorch, and 465.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# --- 4. Validation ---
print("\n--- Starting Validation ---")

print(f"Validating on flowtest: {test_data_path}")
try:
    res_flow = model.val(data=str(test_data_path)) # Use the trained model object
    print("Validation Results (flowtest):")
    # print(res_flow)
except Exception as e:
    print(f"Error during validation on {test_data_path}: {e}")


print(f"\nValidating on watertrashtest: {test_data_path_wt}")
try:
    res_wt = model.val(data=str(test_data_path_wt)) # Use the trained model object
    print("Validation Results (watertrashtest):")
    # print(res_wt)
except Exception as e:
    print(f"Error during validation on {test_data_path_wt}: {e}")